In [1]:
%gui qt
import numpy as np
from matplotlib import cm
import pandas as pd
from analysis.plotting.plot3D import plot3D

import analysis.plotting.plot3D
from importlib import reload

Set-up variables for notebook.

In [214]:
gridSearch_fitTimes = False # Gridsearch or load previous search
ndif_trees, ndif_events = 9, 9 # Number of trees/event points
                               # for grid search
minimise = False # Minimise or use hard-coded value

Get Forest Fit Times Data

In [213]:
if not gridSearch_fitTimes:
    times = pd.read_pickle("forest_fit_times.pkl", compression='infer')
    print("Successfully Unpickled Grid Search.")
else:
    featuress = [['Z_e_m', 'Z_mu_m'],
                 ['m_H', 'Z_e_m',
                  'Z_mu_m', 'delR_Z'],
                 ['Z_e_m', 'Z_mu_m', 'delR_e',
                  'delR_mu', 'delR_Z', 'm_H']]

    numberOfRows = (ndif_trees+1) * (ndif_events+1) * len(featuress)
    print(numberOfRows)
    times = pd.DataFrame(index=np.arange(0, numberOfRows),
                         columns=('trees', 'events', 'features',
                                  'fit_time') )

    n, total_events = 0, len(train_df)
    for trees in np.linspace(1, 500, ndif_trees+1):
        trees = int(trees)

        for events in np.linspace(2, total_events, ndif_events+1):
            frac_events = events / total_events

            for features in featuress:
                n_features = len(features)

                temp = ODataFrame(df[features+['signal']])
                if frac_events < 1:
                    temp = temp.train_test_sets(frac_events)[0]

                forest = RandomForest(n_trees=trees)
                s = time()
                forest.fit(*temp.ML_input(),
                           inform=False, timeout=None)
                f = time()

                times.loc[n] = [trees, len(temp), n_features, f-s]
                n += 1

        print(f"{n:d} Fitted: {trees} trees; " +
              f"{len(temp)} events; {n_features} features")
    
    response = input("Save Grid Search? (y/n): ").lower()
    while response not in ['y', 'yes', 'n', 'no']:
        response = input("Save Grid Search? (y/n): ").lower()
    
    if response in ['y', 'yes']:
        times.to_pickle("forest_fit_times.pkl", protocol=-1)

Successfully Unpickled Grid Search.


Create a dictionary of results by number of features (observables).

In [4]:
gb = times.groupby('features')
features = [gb.get_group(group).reset_index(drop=True)
            for group in gb.groups]

features = {feature.features[0]:feature for feature in features}

for feature in features.values():
    feature.drop(columns='features', inplace=True)

Define a function to estiamte the fit time of a Random Forest. Define a function to returns chi-squared to minimise.

In [191]:
fmin, fmax = np.finfo('float64').min, np.finfo('float64').max

def estimate_surf(n_features, n_trees, n_events, A, a, b, c, d, e):
    estimate = A*(a*n_trees * b*n_features
                  * c*n_events
                  * d*np.log(e*n_events))
    return estimate + 1


def chi_squared(z_true, z_est):
    chisq = (z_true - z_est)**2 / z_true
    return np.sum(chisq)


def sum_chi_squared(z_trues, est_fn, argss):
    sum_chi = 0
    for z_true, args in zip(z_trues, argss):
        z_est = est_fn(*args)
        red_chi = chi_squared(z_true, z_est) / (z_true.size - 1)
        sum_chi += red_chi ** 2
    return max(0, sum_chi**0.5)#, fmax)


def min_fn(coef, wxys, z_trues):
    argss = [tuple([*wxy, *coef]) for wxy in wxys]
    sum_chi = sum_chi_squared(z_trues, estimate_surf, argss)
    return sum_chi

Chi Squared Minimise or use previous 'hard coded' values for coefficents.

In [221]:
wxys, zs = [], []
for n_features, df in features.items():
    df = df.astype(float)\
           .set_index(['trees', 'events'])['fit_time']\
           .unstack(0)
    
    w = n_features
    x, y = np.meshgrid(df.columns, df.index)
    z_true = df.values
    
    wxys.append((w, x, y))
    zs.append(z_true)

initial_guess = (1/4000000, 1, 1, 1, 1 ,1)
sum_chi = min_fn(initial_guess, wxys, zs)
print(f"Initial Guess: {sum_chi:>10,.2f}")

# Minimise Chi-squared for coefficents...
if minimise:
    from scipy.optimize import minimize 
    result = minimize(min_fn, initial_guess, args=(wxys, zs),
                      options={'maxiter':1000, 'disp':True},
                      method='Nelder-Mead')
    best_guess = result.x
# Or use hard-coded results from previous minimisation.
else:
    best_guess = np.array([3.20525123e-07, 1.03130986e+00,
                           1.38976666e+00, 1.57069341e+00,
                           1.08743505e+00, 6.13200046e-03],
                          dtype='float64')
    
sum_chi = min_fn(best_guess, wxys, zs)
print(f"Best Guess: {sum_chi:>13,.2f}")
print("Best Guess Coefficients:", end='')
for n, coef in enumerate(best_guess):
    start = ' [' if (n == 0) else '\t\t\t  '
    end = ' ]\n' if (n+1 == len(best_guess)) else ',\n'
    significand, exponent = f" {coef:.3e}".split('e')
    exponent = f"e{int(exponent):+04d}"
    print(start + significand + exponent, end=end)

print()
for wxy, z in zip(wxys, zs):
    z_est = estimate_surf(*wxy, *best_guess)
    red_chi = chi_squared(z, z_est) / (z.size - 1)
    print(f"Num Features: {wxy[0]} ---> {red_chi:5,.2f}")

Initial Guess:       6.73
Best Guess:          0.67
Best Guess Coefficients: [ 3.205e-007,
			   1.031e+000,
			   1.390e+000,
			   1.571e+000,
			   1.087e+000,
			   6.132e-003 ]

Num Features: 2 --->    0.44
Num Features: 4 --->    0.33
Num Features: 6 --->    0.38


Use coefficents to plot the estimated surfaces and fit times.

In [222]:
reload( analysis.plotting.plot3D )
from analysis.plotting.plot3D import plot3D

sx, sy, sz = 'trees', 'events', 'fit_time'
coefficents = best_guess

z0, z1 = (0, 225)
x0, x1 = (0, max([df[sx].max() for df in features.values()]))
y0, y1 = (0, max([df[sy].max() for df in features.values()]))

ax3d = plot3D(x0=x0, x1=x1, y0=y0, y1=y1, z0=z0, z1=z1)

for n_features, dataframe in features.items():    
    df = dataframe.astype(float).set_index([sx, sy])[sz].unstack(0)

    w = n_features
    x, y = np.meshgrid(df.columns, df.index)
    z = df.values

    ax3d.surf(x, y, z, cmap_props={'alpha':(0.3, 0.9)})

    z_e = estimate_surf(w, x, y, *coefficents) #*1.05
    
    
    ax3d.surf(x, y, z_e, contours={'n_contours': 0},
              colormap='Greens', cmap_props={'alpha':(0.3, 0.9)},
              representation='wireframe')
    
ax3d.x_label="Trees" 
ax3d.y_label="Events" 
ax3d.z_label="Fit Time (s)"

ax3d.view = {'azimuth': 140}

ax3d.show();